In [1]:
import re
import numpy as np
import pandas as pd
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2, SelectKBest, r_regression

In [2]:
def train_model_print_score(model_):
    model_.fit(X_train, Y_train)
    y_pred = model_.predict(X_test)
    print('Классификация без выбора: ')
    print(accuracy_score(Y_test, y_pred))

    model_.fit(X_train_embedded, Y_train)
    y_pred = model_.predict(X_test_embedded)
    print('Классификация после выбора признаков с помощью встроенного метода: ')
    print(accuracy_score(Y_test, y_pred))

    model_.fit(X_train_wrapper, Y_train)
    y_pred = model_.predict(X_test_wrapper)
    print('Классификация после выбора признаков с помощью метода обертки: ')
    print(accuracy_score(Y_test, y_pred))

    model_.fit(X_train_filter, Y_train)
    y_pred = model_.predict(X_test_filter)
    print('Классификация после выбора признаков с помощью фильтрующего метода: ')
    print(accuracy_score(Y_test, y_pred))

    model_.fit(X_train_library_embedded, Y_train)
    y_pred = model_.predict(X_test_library_embedded)
    print('Классификация после выбора признаков с помощью библиотечного встроенного метода: ')
    print(accuracy_score(Y_test, y_pred))

    model_.fit(X_train_library_filter_pearson, Y_train)
    y_pred = model_.predict(X_test_library_filter_pearson)
    print('Классификация после выбора признаков с помощью библиотечного фильтрующего метода с корреляцией Пирсона: ')
    print(accuracy_score(Y_test, y_pred))

    model_.fit(X_train_library_filter_chi2, Y_train)
    y_pred = model_.predict(X_test_library_filter_chi2)
    print('Классификация после выбора признаков с помощью фильтрующего метода c хи-2: ')
    print(accuracy_score(Y_test, y_pred))

In [3]:
ds = pd.read_csv('SMS.tsv', sep='\t')
ds['class'] = pd.factorize(ds['class'])[0]
Y = ds['class']
list_remove = ['\"', ',', ';', ':', '-', '‘', ')', '(', '\'', '$', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
               '—', '*', '/', '–', '.', '?', '!', '#', '&', '»', 'ü', 'ü', '%', '+', '<', '=', '>', '@', '[', '\\', ']',
               '^', '_', '|', '~', '\x91', '\x92', '\x93', '\x94', '\x96', '¡', '£', 'è', 'é', 'ì', 'ú', 'ü', 'é',
               '’', '“', '…', '┾', '〨', '鈥']
text = []
for s in ds['text']:
    for chr1 in list_remove:
        s = s.replace(chr1, '')
    s = re.sub('\s+', ' ', s)
    text.append(s.lower())

vectorizer = TfidfVectorizer(max_features=2000)
X = vectorizer.fit_transform(text)
X = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

# Встроенный метод: RandomForest

In [4]:
X_train_embedded = X_train.copy()
model = RandomForestClassifier()
model.fit(X_train_embedded, Y_train)
embedded_coef = pd.Series(model.feature_importances_, X_train_embedded.columns).sort_values(ascending=False)[:30]
embedded_coef

call          0.045463
txt           0.043858
mobile        0.029138
free          0.029061
claim         0.027062
to            0.018210
stop          0.017222
won           0.016746
your          0.016357
prize         0.015588
text          0.014547
reply         0.013878
win           0.012801
box           0.010831
now           0.010271
from          0.010127
service       0.009877
nokia         0.009323
tone          0.008634
new           0.008418
per           0.008150
customer      0.008086
ringtone      0.007354
contact       0.007272
or            0.007253
cash          0.007056
guaranteed    0.006747
chat          0.006375
pobox         0.006238
pmin          0.006209
dtype: float64

# Метод обертка: SVM RFE

In [5]:
X_train_wrapper = X_train.copy()

while len(X_train_wrapper.columns) > 20:
    model = LinearSVC()
    model.fit(X_train_wrapper, Y_train)
    wrapper_coef = pd.Series(abs(model.coef_[0]), X_train_wrapper.columns).sort_values()
    for i in range(10):
        X_train_wrapper = X_train_wrapper.drop(wrapper_coef.index[i], axis=1)

wrapper_coef

call           1.744378
me             1.939543
text           2.184799
pobox          2.245218
uk             2.306203
apply          2.315629
tone           2.392238
latest         2.408026
landline       2.427568
games          2.476074
tones          2.515200
txts           2.605735
selection      2.692097
video          2.704476
awarded        2.744196
win            2.779752
user           2.810456
reply          2.894941
ltd            2.996169
collection     3.012582
pmin           3.084816
ringtone       3.103743
service        3.206453
unsubscribe    3.432500
mobile         3.456206
won            3.473674
admirer        3.706639
pmsg           3.883373
claim          5.347827
txt            5.390706
dtype: float64

# Фильтрующий метод: Использование корреляции Спирмена

In [6]:
X_train_filter = X_train.copy()
corr = X_train_filter.corr(method='spearman')
corr_class = corr['class']
filter_coef = corr_class.sort_values(ascending=False)[:30]
filter_coef

class          1.000000
willing        0.165239
outta          0.164713
teach          0.164112
derek          0.094457
lr             0.094284
site           0.081455
holder         0.080854
airport        0.080854
five           0.080553
quote          0.071738
using          0.071704
shouldnt       0.071502
hostel         0.071469
came           0.068410
arent          0.064682
wednesday      0.064529
early          0.060077
whos           0.060076
holla          0.059905
coffee         0.059252
midnight       0.059223
wheres         0.055657
especially     0.054939
less           0.054834
yo             0.054088
out            0.052946
water          0.051540
shower         0.051465
information    0.051240
Name: class, dtype: float64

# Первые 30 признаков, выбранные каждым методом

In [7]:
print('Встроенный метод:')
print(np.array(embedded_coef.index))
print()
print('Метод обертка:')
print(np.array(wrapper_coef.index))
print()
print('Фильтрующий метод:')
print(np.array(filter_coef.index))

Встроенный метод:
['call' 'txt' 'mobile' 'free' 'claim' 'to' 'stop' 'won' 'your' 'prize'
 'text' 'reply' 'win' 'box' 'now' 'from' 'service' 'nokia' 'tone' 'new'
 'per' 'customer' 'ringtone' 'contact' 'or' 'cash' 'guaranteed' 'chat'
 'pobox' 'pmin']

Метод обертка:
['call' 'me' 'text' 'pobox' 'uk' 'apply' 'tone' 'latest' 'landline'
 'games' 'tones' 'txts' 'selection' 'video' 'awarded' 'win' 'user' 'reply'
 'ltd' 'collection' 'pmin' 'ringtone' 'service' 'unsubscribe' 'mobile'
 'won' 'admirer' 'pmsg' 'claim' 'txt']

Фильтрующий метод:
['class' 'willing' 'outta' 'teach' 'derek' 'lr' 'site' 'holder' 'airport'
 'five' 'quote' 'using' 'shouldnt' 'hostel' 'came' 'arent' 'wednesday'
 'early' 'whos' 'holla' 'coffee' 'midnight' 'wheres' 'especially' 'less'
 'yo' 'out' 'water' 'shower' 'information']


# Библиотечный встроенный метод: LogisticRegression

In [8]:
X_train_embedded = X_train.copy()
model = LogisticRegression(solver='liblinear', penalty='l1')
model.fit(X_train_embedded, Y_train)
library_embedded_coef = pd.Series(model.coef_[0], X_train_embedded.columns).sort_values(ascending=False)[:30]
library_embedded_coef

txt          14.933139
claim        10.787301
call          7.401519
mobile        7.297542
reply         7.262817
text          7.080068
won           6.743823
chat          5.969300
free          5.841087
win           5.750655
stop          5.694401
ringtone      5.632152
pmin          5.366378
per           5.349392
who           5.166789
tone          4.862679
service       4.855328
pobox         4.821555
box           4.680399
tones         4.514819
from          4.395059
statement     4.057034
sexy          3.791743
games         3.699151
your          3.694103
cash          3.564446
video         3.393181
to            3.385327
real          3.364647
pmsg          3.324321
dtype: float64

# Библиотечный фильтрующий метод с корреляцией Пирсона

In [16]:
X_train_filter = X_train.copy()
model = SelectKBest(r_regression, k=30)
model.fit_transform(X_train_filter, Y_train)
library_filter_pearson_coef = pd.Series(model.scores_, X_train_filter.columns).sort_values(ascending=False)[:30]
library_filter_pearson_coef

txt           0.369703
claim         0.353298
mobile        0.329501
free          0.315083
prize         0.298803
won           0.288424
call          0.280864
urgent        0.253856
stop          0.241448
guaranteed    0.241089
reply         0.240687
win           0.227122
cash          0.225061
service       0.224511
to            0.219672
tone          0.217422
ppm           0.216535
contact       0.215747
nokia         0.213089
our           0.205806
customer      0.200904
awarded       0.200309
box           0.197855
text          0.197156
pobox         0.191157
per           0.188213
or            0.187139
code          0.184904
shows         0.184141
your          0.183885
dtype: float64

# Библиотечный фильтрующий метод с хи-квадрат

In [18]:
X_train_filter = X_train.copy()
model = SelectKBest(chi2, k=30)
model.fit_transform(X_train_filter, Y_train)
library_filter_chi2_coef = pd.Series(model.scores_, X_train_filter.columns).sort_values(ascending=False)[:30]
library_filter_chi2_coef

claim         133.522227
txt           131.717648
free          117.330469
mobile        115.769525
prize         111.317636
won            94.434837
call           81.911938
stop           74.543710
urgent         74.110588
nokia          67.458115
guaranteed     66.388782
tone           63.317491
service        62.562737
reply          61.796344
win            59.233504
cash           58.211015
ppm            53.384260
customer       51.273961
contact        48.688884
ringtone       48.140252
awarded        47.890272
text           45.329725
box            44.010673
our            42.633105
code           42.203637
shows          40.719915
per            40.334973
pmin           40.313433
pobox          40.215500
draw           37.999474
dtype: float64

# Сравнение моих и библиотечных:

In [11]:
print('Мои реализации: ')
print('  1. Встроенный метод')
print(sorted(np.array(embedded_coef.index)))
print('  2. Метод обертка')
print(sorted(np.array(wrapper_coef.index)))
print('  3. Фильтрующий метод')
print(sorted(np.array(filter_coef.index)))
print()
print('Библиотечные: ')
print('  1. Встроенный метод')
print(sorted(np.array(library_embedded_coef.index)))
print('  2. Фильтрующий метод Пирсон')
print(sorted(np.array(library_filter_pearson_coef.index)))
print('  3. Фильтрующий метод хи-2')
print(sorted(np.array(library_filter_chi2_coef.index)))

Мои реализации: 
  1. Встроенный метод
['box', 'call', 'cash', 'chat', 'claim', 'contact', 'customer', 'free', 'from', 'guaranteed', 'mobile', 'new', 'nokia', 'now', 'or', 'per', 'pmin', 'pobox', 'prize', 'reply', 'ringtone', 'service', 'stop', 'text', 'to', 'tone', 'txt', 'win', 'won', 'your']
  2. Метод обертка
['admirer', 'apply', 'awarded', 'call', 'claim', 'collection', 'games', 'landline', 'latest', 'ltd', 'me', 'mobile', 'pmin', 'pmsg', 'pobox', 'reply', 'ringtone', 'selection', 'service', 'text', 'tone', 'tones', 'txt', 'txts', 'uk', 'unsubscribe', 'user', 'video', 'win', 'won']
  3. Фильтрующий метод
['airport', 'arent', 'came', 'class', 'coffee', 'derek', 'early', 'especially', 'five', 'holder', 'holla', 'hostel', 'information', 'less', 'lr', 'midnight', 'out', 'outta', 'quote', 'shouldnt', 'shower', 'site', 'teach', 'using', 'water', 'wednesday', 'wheres', 'whos', 'willing', 'yo']

Библиотечные: 
  1. Встроенный метод
['box', 'call', 'cash', 'chat', 'claim', 'free', 'from', 

# Выбранные классификаторы: kNN, SVM, DecisionTree

In [12]:
X_train_embedded = pd.DataFrame(X_train, columns=np.array(embedded_coef.index))
X_test_embedded = pd.DataFrame(X_test, columns=np.array(embedded_coef.index))

X_train_wrapper = pd.DataFrame(X_train, columns=np.array(wrapper_coef.index))
X_test_wrapper = pd.DataFrame(X_test, columns=np.array(wrapper_coef.index))

X_train_filter = pd.DataFrame(X_train, columns=np.array(filter_coef.index))
X_test_filter = pd.DataFrame(X_test, columns=np.array(filter_coef.index))

X_train_library_embedded = pd.DataFrame(X_train, columns=np.array(library_embedded_coef.index))
X_test_library_embedded = pd.DataFrame(X_test, columns=np.array(library_embedded_coef.index))

X_train_library_filter_pearson = pd.DataFrame(X_train, columns=np.array(library_filter_pearson_coef.index))
X_test_library_filter_pearson = pd.DataFrame(X_test, columns=np.array(library_filter_pearson_coef.index))

X_train_library_filter_chi2 = pd.DataFrame(X_train, columns=np.array(library_filter_chi2_coef.index))
X_test_library_filter_chi2 = pd.DataFrame(X_test, columns=np.array(library_filter_chi2_coef.index))

In [13]:
print('KNN: ')
train_model_print_score(KNeighborsClassifier())

KNN: 
Классификация без выбора: 
0.9461883408071748
Классификация после выбора признаков с помощью встроенного метода: 
0.9587443946188341
Классификация после выбора признаков с помощью метода обертки: 
0.9650224215246637
Классификация после выбора признаков с помощью фильтрующего метода: 
0.8914798206278027
Классификация после выбора признаков с помощью библиотечного встроенного метода: 
0.957847533632287
Классификация после выбора признаков с помощью библиотечного фильтрующего метода с корреляцией Пирсона: 
0.9605381165919282
Классификация после выбора признаков с помощью фильтрующего метода c хи-2: 
0.9623318385650225


In [14]:
print('SVM: ')
train_model_print_score(SVC())

SVM: 
Классификация без выбора: 
0.9865470852017937
Классификация после выбора признаков с помощью встроенного метода: 
0.9641255605381166
Классификация после выбора признаков с помощью метода обертки: 
0.9614349775784753
Классификация после выбора признаков с помощью фильтрующего метода: 
0.8914798206278027
Классификация после выбора признаков с помощью библиотечного встроенного метода: 
0.9632286995515695
Классификация после выбора признаков с помощью библиотечного фильтрующего метода с корреляцией Пирсона: 
0.9695067264573991
Классификация после выбора признаков с помощью фильтрующего метода c хи-2: 
0.9641255605381166


In [15]:
print('Decision Tree: ')
train_model_print_score(DecisionTreeClassifier())

Decision Tree: 
Классификация без выбора: 
0.9632286995515695
Классификация после выбора признаков с помощью встроенного метода: 
0.9524663677130045
Классификация после выбора признаков с помощью метода обертки: 
0.9596412556053812
Классификация после выбора признаков с помощью фильтрующего метода: 
0.8896860986547085
Классификация после выбора признаков с помощью библиотечного встроенного метода: 
0.9560538116591928
Классификация после выбора признаков с помощью библиотечного фильтрующего метода с корреляцией Пирсона: 
0.9488789237668162
Классификация после выбора признаков с помощью фильтрующего метода c хи-2: 
0.9506726457399103
